Primero, clonaremos tu repositorio de GitHub en el entorno de Colab:

In [ ]:
!git clone https://github.com/ichiP245/micrography-imgpro.git

Luego, agregaremos la carpeta del repositorio al `sys.path` de Python para que podamos importar el módulo `getmeresults`.

In [ ]:
import sys
sys.path.append('/content/micrography-imgpro')

# Verifica que el módulo esté en la ruta antes de intentar importarlo
print("Python path updated. Trying to import getmeresults...")

In [ ]:
# Antes de importar, asegúrate de que el módulo esté instalado.
# Si 'getmeresults' es un paquete de Python, puedes intentar instalarlo así:
# !pip install getmeresults
# Si es un archivo local o un módulo personalizado, asegúrate de que esté en el PYTHONPATH o en la misma carpeta.
import getmeresults as gmr

In [ ]:
import cv2
import matplotlib.pyplot as plt
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Carpeta en Google Drive -> Tubo 2024
folder_path = '/content/drive/MyDrive/Micrografía M2 31 3 2025'
list_path_imgs = []
for item_name in os.listdir(folder_path):
    item_path = os.path.join(folder_path, item_name)
    if os.path.isfile(item_path):
        if len(item_name) < 22:
          print(f"File: {item_name} not added")
        else:
          list_path_imgs.append(item_path)
    elif os.path.isdir(item_path):
        print(f"Directory: {item_name}")

In [ ]:
def read_and_crop_image(path):
    import cv2, numpy as np
    img = cv2.imread(path)
    h, w = img.shape[:2]
    img = img[:h-290, :, :]  # or use auto-detection like above
    return img

In [ ]:
for img_path in list_path_imgs:
    # Read the original image
    original_img = cv2.imread(img_path)

    # Get the cropped image using the defined function
    cropped_img = read_and_crop_image(img_path)

    # Convert images from BGR to RGB for matplotlib display
    original_img_rgb = cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB)
    cropped_img_rgb = cv2.cvtColor(cropped_img, cv2.COLOR_BGR2RGB)

    # Display original and cropped images side-by-side
    plt.figure(figsize=(15, 7))

    plt.subplot(1, 2, 1) # 1 row, 2 columns, first plot
    plt.imshow(original_img_rgb)
    plt.title(f"Original: {os.path.basename(img_path)}")
    plt.axis('off')

    plt.subplot(1, 2, 2) # 1 row, 2 columns, second plot
    plt.imshow(cropped_img_rgb)
    plt.title("Recortada")
    plt.axis('off')

    plt.show()

# Todas las imagenes quedaron bien recortadas

In [ ]:
list_cropped_imgs = []
for img_path in list_path_imgs:
  # Recortamos la imagen
  cropped_img = read_and_crop_image(img_path)
  cropped_img_rgb = cv2.cvtColor(cropped_img, cv2.COLOR_RGB2GRAY)
  list_cropped_imgs.append(cropped_img_rgb)

In [ ]:
global_parameters = {
    'first_kernel_size': (5,5),
    'second_kernel_size': (3,3),
    'contours_mult': 2.5,
    'bh_ks': (7,7),
    'bhm_iter': 4,
    'bhm_mult': 60,
    'cont_mult': 2.5,
    'ws_ths_factor': 0.025,
    'ws_gl_vecinity': 15}

In [ ]:
dict_results = dict()

In [ ]:
for i in range(3):
  print(f"Image: {list_path_imgs[i]}")
  path_image = list_path_imgs[i]
  grayscale_image = list_cropped_imgs[i]
  perc, seg, col = gmr.getMeResults(grayscale_image, global_parameters)
  dict_results[path_image] = [perc, seg, col]

In [ ]:
import cv2

# Convert the input image to grayscale
grayscale_image = cv2.cvtColor(list_cropped_imgs[0], cv2.COLOR_RGB2GRAY)

perc, seg, col = gmr.getMeResults(grayscale_image, global_parameters)

In [ ]:
def getSegmentationFigure(segmentation, percentages, filename):
    fig = plt.figure(figsize=(14, 8))
    plt.imshow(segmentation)
    plt.axis('off')

    # Agregar la barra de color con etiquetas personalizadas
    cbar = plt.colorbar(orientation="vertical",shrink=0.4)
    cbar.set_ticks([20,60,150,250])
    cbar.set_ticklabels([
        f"Indefinido - {percentages['undefined']:.2f}%",
        f"Poros - {percentages['pores']:.2f}%",
        f"Resina - {percentages['resin']:.2f}%",
        f"Fibra - {percentages['fibers']:.2f}%"
        ])
    plt.title(f"Segmentación - {filename}")
    plt.tight_layout()

In [ ]:
for path, results in dict_results.items():
  perc, seg, col = results
  getSegmentationFigure(seg, perc, os.path.basename(path))